# Visual anchor selector based on bounds

A rudimentary helper for selecting vertices on a mesh.

In [1]:
import json
import numpy as np
import meshplot as mp
from scripts import load_mesh, compute_laplacian, compute_massmatrix, \
    compute_eigenv_sparse

## Select the vertices

This program uses bounds on coordinate dimensions to select a group of vertices.

For manual picking, you may prefer the `selectanchors.py` command line script.

### Instructions

1. First, set the mesh file path using `mesh_file`.
2. Then, to help finding vertex coordinate bounds, use the interactive sliders to draw points on the mesh plot. You can set the sliders range with `slider_range`.
3. Finally, copy the bounds values to `x`, `y`, `z`, and select the direction to include:
    - greater than == True
    - less than == False

In [2]:
# Set mesh file and sliders range.
mesh_file = "input/armadillo_1k.off"
slider_range = (-1., 1., 0.0001)

# Load and display mesh
v, f = load_mesh(mesh_file)
d = mp.plot(v, f, c=v[:, 1], return_plot=True)

# Create interactive sliders
def f(x, y, z):
    pts = np.array([[x, y, z]])
    d.add_points(pts, shading={"point_color": "red", "point_size": 0.1})
    return x

mp.interact(f, x=slider_range, y=slider_range, z=slider_range)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0138960…

interactive(children=(FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.0001), FloatSlider(val…

<function __main__.f(x, y, z)>

In [6]:
# Set selection boundary
# Selection 1
x = 0.30
increasing_x = True

y = 0.26
increasing_y = True

z = -0.18
increasing_z = False

# # Selection 2
# x = -9999
# increasing_x = True

# y = -0.48
# increasing_y = False

# z = 9999
# increasing_z = False

# Select vertices
def select_vertices(axis, value, vertex_set, direction):
    """ Select vertices along one axis according to specified value and direction.
    
    Args:
        axis (string): the axis
        value (float): the bound of the selection
        vertex_set (np.ndarray): the set of vertices from which to select
        direction (bool): the direction, True = higher, False = lower
    
    Returns:
        np.ndarray: the selected vertices
    """
    axes = {
        "x": 0,
        "y": 1,
        "z": 2
    }
    axis = axes[axis]
    
    indices = []
    vertices = []
    
    for index, vertex in enumerate(vertex_set):
        if direction:
            if vertex[axis] > value:
                indices.append(index)
                vertices.append(vertex)
        else:
            if vertex[axis] < value:
                indices.append(index)
                vertices.append(vertex)
                                
    return indices, vertices

# Find the vertices
indices_1, select_by_x = select_vertices("x", x, v, increasing_x)
indices_2, select_by_y = select_vertices("y", y, select_by_x, increasing_y)
indices_3, selected = select_vertices("z", z, select_by_y, increasing_z)

for i in range(len(selected)):
    selected[i] = selected[i].tolist()

# Retrieve the indices
indices = [indices_1[i1] for i1 in [indices_2[i2] for i2 in indices_3]]

## Save the selection to make another one

In [7]:
# Selections
selection_1_indices = indices
selection_1_selected = selected

# selection_2_indices = indices
# selection_2_selected = selected

In [8]:
# Check selections
print(selection_1_indices)
print(selection_2_indices)

[670, 679, 688, 693, 710, 729, 730, 746, 754, 761, 762, 770, 775, 781, 784, 788, 795, 803, 808, 809, 816, 821, 822, 825]
[0, 17, 18, 27, 29, 36, 37, 47, 48, 49, 64, 66, 67, 81, 82, 93, 94, 98, 105, 108, 109, 110, 121, 122, 137, 139, 152, 153, 179, 181, 188, 189, 204, 206, 207, 209, 211, 216, 229, 231, 232, 235, 239, 240, 241, 284, 288, 289, 291, 293, 296, 300, 307, 316, 324, 328, 330, 346, 356, 380, 401, 412, 431, 433, 470, 507, 518, 571, 574, 580, 584, 601, 606, 641, 677, 682, 720, 740, 743, 794, 818]


## Move the choosen vertices

In [9]:
# backup = selection_1_selected
selection_1_moved = selection_1_selected + np.array([1, 1, 0])

In [10]:
# Check movement
print(selection_1_selected)
print(selection_1_moved)

[[0.367542, 0.35022, -0.185912], [0.334203, 0.311586, -0.18917], [0.38234, 0.343596, -0.201376], [0.427481, 0.265231, -0.182077], [0.425766, 0.284506, -0.209741], [0.345912, 0.348994, -0.219485], [0.356636, 0.29111, -0.223968], [0.395689, 0.323604, -0.23446], [0.337906, 0.379843, -0.208252], [0.345978, 0.329238, -0.235828], [0.42547, 0.270606, -0.241654], [0.363501, 0.330691, -0.257165], [0.408756, 0.296902, -0.2511], [0.376021, 0.354701, -0.244533], [0.363407, 0.310052, -0.257869], [0.378165, 0.293864, -0.259641], [0.392787, 0.281106, -0.266187], [0.348884, 0.366952, -0.246671], [0.345315, 0.352291, -0.262713], [0.396219, 0.31758, -0.270987], [0.377043, 0.333428, -0.272238], [0.400503, 0.291763, -0.27438], [0.402826, 0.264528, -0.274417], [0.360786, 0.316454, -0.285082]]
[[ 1.367542  1.35022  -0.185912]
 [ 1.334203  1.311586 -0.18917 ]
 [ 1.38234   1.343596 -0.201376]
 [ 1.427481  1.265231 -0.182077]
 [ 1.425766  1.284506 -0.209741]
 [ 1.345912  1.348994 -0.219485]
 [ 1.356636  1.2911

## Output to JSON file

Run this cell to save the selected vertices to a JSON file.

In [43]:
mesh_name = mesh_file.split(".")[0].split("/")[-1]
output_file = f"output/{mesh_name}-anchors.json"

# Combine selections
selection_1 = list(zip(selection_1_indices, selection_1_moved))
selection_2 = list(zip(selection_2_indices, selection_2_selected))
selection_merged = np.concatenate((selection_1, selection_2))

# Convert to Python list
selection_merged = np.array(selection_merged)
for i in range(len(selection_merged)):
    if type(selection_merged[i][1]).__module__ == np.__name__:
        selection_merged[i][1] = selection_merged[i][1].tolist()

# Reorder based on indices
selection_merged = selection_merged[selection_merged[:, 0].argsort()]


# Save selected vertices as JSON
data = {
    "indices": selection_merged[:, 0].tolist(),
    "new_positions": selection_merged[:, 1].tolist()
}

with open(output_file, "w+") as f:
    json.dump(data, f)